# LAB 4 : FP Growth Algorithm in Data Mining

In Data Mining, finding frequent patterns in large databases is very important and has been studied on a large scale in the past few years. Unfortunately, this task is computationally expensive, especially when many patterns exist.

The FP-Growth Algorithm proposed by Han in is an efficient and scalable method for mining the complete set of frequent patterns by pattern fragment growth, using an extended prefix-tree structure for storing compressed and crucial information about frequent patterns named frequent-pattern tree (FP-tree). In his study, Han proved that his method outperforms other popular methods for mining frequent patterns, e.g., the Apriori Algorithm and the TreeProjection. In some later works, it was proved that FP-Growth performs better than other methods, including Eclat and Relim. The popularity and efficiency of the FP-Growth Algorithm contribute to many studies that propose variations to improve its performance.

## What is FP Growth Algorithm?

The FP-Growth Algorithm is an alternative way to find frequent item sets without using candidate generations, thus improving performance. For so much, it uses a divide-and-conquer strategy. The core of this method is the usage of a special data structure named frequent-pattern tree (FP-tree), which retains the item set association information.

This algorithm works as follows:

1. First, it compresses the input database creating an FP-tree instance to represent frequent items.
2. After this first step, it divides the compressed database into a set of conditional databases, each associated with one frequent pattern.
3. Finally, each such database is mined separately.
  
Using this strategy, the FP-Growth reduces the search costs by recursively looking for short patterns and then concatenating them into the long frequent patterns.

In large databases, holding the FP tree in the main memory is impossible. A strategy to cope with this problem is to partition the database into a set of smaller databases (called projected databases) and then construct an FP-tree from each of these smaller databases.

### FP-Growth Algorithm

**Input:** 

- Database \( D \) of transactions.
- Minimum support threshold \( \text{min_sup} \).

**Output:**

- Set of frequent itemsets.

**Steps:**

1. **Scan Database and Count Item Occurrences:**
   - Scan the database to determine the frequency of each item.
   - Filter out infrequent items that do not meet the minimum support threshold.
   - Sort the frequent items in decreasing order of frequency.

2. **Construct the FP-Tree:**
   - Initialize an empty FP-Tree.
   - For each transaction in the database:
     - Sort the frequent items in the transaction according to their frequency.
     - Add the sorted transaction to the FP-Tree while preserving the item order.
     - Update the header table to link identical items in the FP-Tree.

3. **Mine FP-Tree for Frequent Patterns:**
   - For each frequent item in the header table (starting from the least frequent):
     - Initialize an empty conditional pattern base.
     - For each path in the FP-Tree that contains the current item:
       - Extract the prefix path and its count from the FP-Tree.
       - Construct the conditional pattern base by concatenating the prefix path with the current item.
       - Recursively mine the conditional FP-Tree formed by the conditional pattern base.
       - Combine the frequent patterns generated from the conditional FP-Tree with the current item to form new frequent patterns.

4. **Return Frequent Patterns:**
   - Return the set of frequent patterns generated from the FP-Tree mining process.

## Advantages of FP Growth Algorithm

Here are the following advantages of the FP growth algorithm:

- This algorithm needs to scan the database twice when compared to Apriori, which scans the transactions for each iteration.
- The pairing of items is not done in this algorithm, making it faster.
- The database is stored in a compact version in memory.
- It is efficient and scalable for mining both long and short frequent patterns.

## Disadvantages of FP-Growth Algorithm

This algorithm also has some disadvantages, such as:

- FP Tree is more cumbersome and difficult to build than Apriori.
- It may be expensive.
- The algorithm may not fit in the shared memory when the database is large.

## Difference between Apriori and FP Growth Algorithm
| Feature                                                     | Apriori                                    | FP-Growth                                 |
|:----------------------------------------------------------- |:------------------------------------------ |:----------------------------------------- |
| Generation of Frequent Patterns                             | Uses itemset pairings (single, double, etc.) | Utilizes FP-Tree structure                |
| Candidate Generation                                        | Extends frequent subsets one item at a time | Generates conditional FP-Tree for each item |
| Database Scanning                                          | Requires multiple scans, time-consuming for large datasets | Requires only one scan, faster performance |
| Memory Consumption                                         | Stores converted database in memory        | Saves set of conditional FP-Trees in memory |
| Search Strategy                                            | Utilizes breadth-first search               | Employs depth-first search                |


# IMPLEMENTATION

# 1.Install the required libraries if you haven't already:

In [1]:
# !pip install mlxtend

# 2.Import the necessary libraries:

In [2]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules
import pandas as pd


# 3.Define your dataset:

In [3]:
dataset = [['I1', 'I2', 'I3'],
           ['I2', 'I3', 'I4'],
           ['I4', 'I5'],
           ['I1', 'I2', 'I4'],
           ['I1', 'I2', 'I3', 'I5'],
           ['I1', 'I2', 'I3', 'I4']]


# 4.Convert the dataset to one-hot encoded format:

In [4]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)


# 5.Perform frequent pattern mining using FP-growth algorithm:

In [5]:
frequent_patterns = fpgrowth(df, min_support=0.5, use_colnames=True)


# 6.Define functions to generate conditional pattern base and conditional FP-tree:

In [6]:
# Function to generate conditional pattern base
def generate_conditional_pattern_base(itemset, dataset):
    conditional_pattern_base = []
    for transaction in dataset:
        if set(itemset).issubset(set(transaction)):
            conditional_pattern_base.append(transaction)
    return conditional_pattern_base


In [7]:
# Function to generate conditional FP-tree
def generate_conditional_fp_tree(conditional_pattern_base):
    te = TransactionEncoder()
    te_ary = te.fit(conditional_pattern_base).transform(conditional_pattern_base)
    conditional_fp_tree = pd.DataFrame(te_ary, columns=te.columns_)
    return conditional_fp_tree

# 7.Define a function to display frequent patterns and generate associative rules:

In [8]:
# Function to display frequent patterns and generate associative rules
def display_frequent_patterns_and_associative_rules(dataset, frequent_patterns):
    table_data = []
    for itemset in frequent_patterns['itemsets']:
        itemset_str = ', '.join(itemset)
        conditional_pattern_base = generate_conditional_pattern_base(itemset, dataset)
        conditional_fp_tree = generate_conditional_fp_tree(conditional_pattern_base)
        frequent_patterns_generated_str = frequent_patterns[frequent_patterns['itemsets'].apply(lambda x: set(x).issuperset(set(itemset)))]
        frequent_patterns_generated_str = ', '.join([str(set(pattern)) for pattern in frequent_patterns_generated_str['itemsets']])
        table_data.append([itemset_str, conditional_pattern_base, conditional_fp_tree, frequent_patterns_generated_str])

    df = pd.DataFrame(table_data, columns=['Item', 'Conditional Pattern Base', 'Conditional FP-tree', 'Frequent Patterns Generated'])
    display(df)

    # Generate associative rules
    rules = association_rules(frequent_patterns, metric="confidence", min_threshold=0.6)
    print("\nAssociation Rules:")
    print(rules[['antecedents', 'consequents', 'support', 'confidence']])


# 8.Call the function to display frequent patterns and generate associative rules:

In [9]:
display_frequent_patterns_and_associative_rules(dataset, frequent_patterns)


,Item,Conditional Pattern Base,Conditional FP-tree,Frequent Patterns Generated
0,I2,"[[I1, I2, I3], [I2, I3, I4], [I1, I2, I4], [I1...",I1 I2 I3 I4 I5 0 True ...,"{'I2'}, {'I2', 'I3'}, {'I2', 'I1'}, {'I2', 'I1..."
1,I3,"[[I1, I2, I3], [I2, I3, I4], [I1, I2, I3, I5],...",I1 I2 I3 I4 I5 0 True T...,"{'I3'}, {'I2', 'I3'}, {'I1', 'I3'}, {'I2', 'I1..."
2,I1,"[[I1, I2, I3], [I1, I2, I4], [I1, I2, I3, I5],...",I1 I2 I3 I4 I5 0 True Tr...,"{'I1'}, {'I2', 'I1'}, {'I1', 'I3'}, {'I2', 'I1..."
3,I4,"[[I2, I3, I4], [I4, I5], [I1, I2, I4], [I1, I2...",I1 I2 I3 I4 I5 0 False ...,"{'I4'}, {'I2', 'I4'}"
4,"I2, I3","[[I1, I2, I3], [I2, I3, I4], [I1, I2, I3, I5],...",I1 I2 I3 I4 I5 0 True T...,"{'I2', 'I3'}, {'I2', 'I1', 'I3'}"
5,"I2, I1","[[I1, I2, I3], [I1, I2, I4], [I1, I2, I3, I5],...",I1 I2 I3 I4 I5 0 True Tr...,"{'I2', 'I1'}, {'I2', 'I1', 'I3'}"
6,"I1, I3","[[I1, I2, I3], [I1, I2, I3, I5], [I1, I2, I3, ...",I1 I2 I3 I4 I5 0 True Tru...,"{'I1', 'I3'}, {'I2', 'I1', 'I3'}"
7,"I2, I1, I3","[[I1, I2, I3], [I1, I2, I3, I5], [I1, I2, I3, ...",I1 I2 I3 I4 I5 0 True Tru...,"{'I2', 'I1', 'I3'}"
8,"I2, I4","[[I2, I3, I4], [I1, I2, I4], [I1, I2, I3, I4]]",I1 I2 I3 I4 0 False True T...,"{'I2', 'I4'}"



Association Rules:
   antecedents consequents   support  confidence
0         (I2)        (I3)  0.666667        0.80
1         (I3)        (I2)  0.666667        1.00
2         (I2)        (I1)  0.666667        0.80
3         (I1)        (I2)  0.666667        1.00
4         (I1)        (I3)  0.500000        0.75
5         (I3)        (I1)  0.500000        0.75
6     (I2, I1)        (I3)  0.500000        0.75
7     (I2, I3)        (I1)  0.500000        0.75
8     (I1, I3)        (I2)  0.500000        1.00
9         (I2)    (I1, I3)  0.500000        0.60
10        (I1)    (I2, I3)  0.500000        0.75
11        (I3)    (I2, I1)  0.500000        0.75
12        (I2)        (I4)  0.500000        0.60
13        (I4)        (I2)  0.500000        0.75


In [10]:
    # Generate associative rules
    rules = association_rules(frequent_patterns, metric="confidence", min_threshold=0.6)
    print("\nAssociation Rules:")
    print(rules[['antecedents', 'consequents', 'support', 'confidence']])


Association Rules:
   antecedents consequents   support  confidence
0         (I2)        (I3)  0.666667        0.80
1         (I3)        (I2)  0.666667        1.00
2         (I2)        (I1)  0.666667        0.80
3         (I1)        (I2)  0.666667        1.00
4         (I1)        (I3)  0.500000        0.75
5         (I3)        (I1)  0.500000        0.75
6     (I2, I1)        (I3)  0.500000        0.75
7     (I2, I3)        (I1)  0.500000        0.75
8     (I1, I3)        (I2)  0.500000        1.00
9         (I2)    (I1, I3)  0.500000        0.60
10        (I1)    (I2, I3)  0.500000        0.75
11        (I3)    (I2, I1)  0.500000        0.75
12        (I2)        (I4)  0.500000        0.60
13        (I4)        (I2)  0.500000        0.75
